In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from data_preprocessing import Autodata
from datasets import concatenate_datasets
from sklearn.model_selection import train_test_split
from dataloader import CustomDataset
from utils import compute_metrics

In [ ]:
model_name = "monologg/koelectra-small-v3-discriminator"
data = Autodata("./data")
cfn_question = data.concat_dataset["question"]
normal_instruction_question = data.load_instruction_dataset("nlpai-lab/kullm-v2")

In [10]:
normal_instruction_question = normal_instruction_question[:1000]

In [ ]:
cfn_data = data.label_indexing(cfn_question, state=0)
normal_instruction_data = data.label_indexing(normal_instruction_question, state=1)

total_data = concatenate_datasets([cfn_data, normal_instruction_data])
train_dataset, val_dataset = train_test_split(
    total_data, test_size=0.2, random_state=42
)

In [ ]:
train_data = CustomDataset(
    data_file=train_dataset,
    model_name=model_name,
    text_columns="question",
    target_columns="target",
    max_length=256,
    state="train",
)

val_data = CustomDataset(
    data_file=val_dataset,
    model_name=model_name,
    text_columns="question",
    target_columns="target",
    max_length=256,
    state="train",
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, ignore_mismatched_sizes=True
)

In [ ]:
args = TrainingArguments(
    output_dir="output_dir",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    dataloader_num_workers=4,
    logging_steps=50,
    seed=42,
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

trainer.train()